In [2]:
import os
import numpy as np
import torch
import glob
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torchvision
import pathlib


device=torch.device('cuda' if torch.cuda.is_available() else'cpu')
transformer=transforms.Compose([
    transforms.Resize([150,150]),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.5,0.5,0.5],
                       [0.5,0.5,0.5])
])


train_path='train'
test_path='test'


train_loader=DataLoader(
    torchvision.datasets.ImageFolder(train_path,transform=transformer),
    batch_size=256,shuffle=True
)

test_loader=DataLoader(
    torchvision.datasets.ImageFolder(test_path,transform=transformer),
    batch_size=256,shuffle=True
)


root=pathlib.Path(train_path)
classes=sorted([j.name.split('/')[-1] for j in root.iterdir()])

class ConvNet(nn.Module):
    def __init__(self,num_classes=6):
        super(ConvNet,self).__init__()
        self.conv1=nn.Conv2d(in_channels=3,out_channels=12,kernel_size=3,stride=1,padding=1)
        self.bn1=nn.BatchNorm2d(num_features=12)
        self.relu1=nn.ReLU()
        self.pool=nn.MaxPool2d(kernel_size=2)
        self.conv2=nn.Conv2d(in_channels=12,out_channels=20,kernel_size=3,stride=1,padding=1)
        self.relu2=nn.ReLU()
        self.conv3=nn.Conv2d(in_channels=20,out_channels=32,kernel_size=3,stride=1,padding=1)
        self.bn3=nn.BatchNorm2d(num_features=32)
        self.relu3=nn.ReLU()
        self.fc=nn.Linear(in_features=75 * 75 * 32,out_features=num_classes)
      
    def forward(self,input):
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)
        output=self.pool(output)
        output=self.conv2(output)
        output=self.relu2(output)
        output=self.conv3(output)
        output=self.bn3(output)
        output=self.relu3(output) 
        output=output.view(-1,32*75*75)  
        output=self.fc(output)
        return output
    
model=ConvNet(num_classes=6).to(device)
optimizer=Adam(model.parameters(),lr=0.001,weight_decay=0.0001)
loss_function=nn.CrossEntropyLoss()
num_epochs=10
train_count=len(glob.glob(train_path+'/**/*.jpeg'))
test_count=len(glob.glob(test_path+'/**/*.jpeg'))
best_accuracy=0.0
for epoch in range(num_epochs):
    model.train()
    train_accuracy=0.0
    train_loss=0.0
    for i, (images,labels) in enumerate(train_loader):
        if torch.cuda.is_available():
            images=Variaable(image.cuda())
            labels=Variable(labels.cuda())
        optimizer.zero_grad()
        outputs=model(images)
        loss=loss_function(outputs,labels)
        loss.backward()
        optimizer.step()
        
        train_loss+=loss.cpu().data*images.size(0)
        _,pridiction=torch.max(outputs.data,1)
        train_accuracy+=int(torch.sum(pridiction==labels.data))
    train_accuracy=train_accuracy/train_count
    train_loss=train_loss/train_count
        
        
        
    model.eval()
    test_accuracy=0.0
 
    for i, (images,labels) in enumerate(test_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
        outputs=model(images)
    
        _,pridiction=torch.max(outputs.data,1)
        test_accuracy+=int(torch.sum(pridiction==labels.data))
    test_accuracy=test_accuracy/test_count
    print('Epoch:'+str(epoch)+'Train Loss:'+str(int(train_loss))+'Train Accuracy:'+str(train_accuracy)+'Test Accuracy:'+str(test_accuracy))
    if test_accuracy>best_accuracy:
        torch.save(model.state_dict(),'checkpoint.model')
        best_accuracy==test_accuracy

Epoch:0Train Loss:1Train Accuracy:0.297029702970297Test Accuracy:0.0
Epoch:1Train Loss:22Train Accuracy:0.7722772277227723Test Accuracy:0.0
Epoch:2Train Loss:17Train Accuracy:0.7722772277227723Test Accuracy:0.0
Epoch:3Train Loss:6Train Accuracy:0.801980198019802Test Accuracy:0.0
Epoch:4Train Loss:15Train Accuracy:0.5346534653465347Test Accuracy:0.0
Epoch:5Train Loss:6Train Accuracy:0.6435643564356436Test Accuracy:0.0
Epoch:6Train Loss:5Train Accuracy:0.8217821782178217Test Accuracy:0.0
Epoch:7Train Loss:6Train Accuracy:0.7821782178217822Test Accuracy:0.0
Epoch:8Train Loss:5Train Accuracy:0.8613861386138614Test Accuracy:0.0
Epoch:9Train Loss:5Train Accuracy:0.8514851485148515Test Accuracy:0.0
